In [1]:
import sys
import numpy as np
import cv2
from time import sleep
from datetime import datetime
import pysolar
import os
import numpy as np
from random import shuffle
from matplotlib import pyplot as plt
import matplotlib as mpl
from pandas import read_csv, read_excel, DataFrame

from skimage.feature import local_binary_pattern as LBP
#from sklearn.model_selection import train_test_split
from dask_ml.model_selection import train_test_split
#from sklearn.experimental import enable_halving_search_cv # noqa
#from sklearn.model_selection import HalvingGridSearchCV
from dask_ml.model_selection import SuccessiveHalvingSearchCV as HalvingGridSearchCV

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
#from sklearn.preprocessing import StandardScaler
from dask_ml.preprocessing import StandardScaler
#from sklearn.decomposition import PCA
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline
from utils_segmentation import get_features

from dask.distributed import Client, progress, LocalCluster
import dask.array as da 

import pickle
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
cl=Client()
cl

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 1.46 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33689,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 1.46 TiB
Comm: tcp://127.0.0.1:33171,Total threads: 4
Dashboard: http://127.0.0.1:37823/status,Memory: 186.75 GiB
Nanny: tcp://127.0.0.1:43823,


In [3]:
version = 'V1'
model = 'mlp'
print(version)
p1 = os.path.join('../../','work','CookIRCamET','Images','CookHY2023',version,'TifPng','RGB')
p2 = os.path.join('../../','work','CookIRCamET','Images','CookHY2023',version,'TifPng')
p11 = os.path.join('../../','work','CookIRCamET','Images','CprlHY2023',version,'TifPng','RGB')
p22 = os.path.join('../../','work','CookIRCamET','Images','CprlHY2023',version,'TifPng')

V1


In [4]:
f_imgs=[]
n_img=0

feats_raw = []
labels_raw = []
for di,do in zip([p1,p11],[p2,p22]):
    fs=os.listdir(di)
    print(di)
    print(do)
    shuffle(fs)
    for f in fs:
        if 'bgr' in f:

            f_imgs = np.append(f_imgs,f)
            print(f)
            filepath = os.path.join(di,f)
            bgr = cv2.imread(filepath,cv2.IMREAD_UNCHANGED)
            time_place = f.split('_bgr.')[0].split('_')

            labels = False
            f_labels = os.path.join(do,'SunShade',f.split('_bgr')[0]+'_class2.tif')
            if (os.path.exists(f_labels)):
                labels1 = cv2.imread(f_labels,cv2.IMREAD_UNCHANGED)
                labels = True

            f_labels = os.path.join(do,'SoilResVegSnow',f.split('_bgr')[0]+'_class4.tif')
            if (os.path.exists(f_labels)):
                labels2 = cv2.imread(f_labels,cv2.IMREAD_UNCHANGED)
                labels2[labels2==4]=3#flowers->veg
                labels = (True & labels)
                if labels: 
                    #8-class
                    labels3 = 4*labels1+labels2
                    if not os.path.exists(os.path.join(do,'Masks')): os.mkdir(os.path.join(do,'Masks'))
                    cv2.imwrite(os.path.join(do,'Masks',f.split('_bgr')[0]+'_class8.png'),labels3)

                feat = get_features(bgr)
                labels1 = labels1.ravel()        
                labels2 = labels2.ravel() 
                labels3 = labels3.ravel() 

                if not np.any(np.isnan(feat)):
                    feats_raw.append(da.from_array(feat))
                    labels_raw.append(da.from_array(labels3))
                    n_img=n_img+1

n_feat = feat.shape[1]


../../work/CookIRCamET/Images/CookHY2023/V1/TifPng/RGB
../../work/CookIRCamET/Images/CookHY2023/V1/TifPng
20221028231220_-117.081888_46.781508_bgr.png
20221028234226_-117.081888_46.781508_bgr.png
20221102223348_-117.081888_46.781508_bgr.png
20230109215441_-117.081797_46.781650_bgr.png
20221102213337_-117.081888_46.781508_bgr.png
20221028224215_-117.081888_46.781508_bgr.png
20221029161523_-117.081888_46.781508_bgr.png
20221205153934_-117.081863_46.781553_bgr.png
20221102233359_-117.081888_46.781508_bgr.png
20221103160656_-117.081888_46.781508_bgr.png
20221029194602_-117.081888_46.781508_bgr.png
20221029144507_-117.081888_46.781508_bgr.png
20221029211618_-117.081888_46.781508_bgr.png
20221102153232_-117.081888_46.781508_bgr.png
20221029204613_-117.081888_46.781508_bgr.png
20230110152751_-117.081797_46.781650_bgr.png
20221102200321_-117.081888_46.781508_bgr.png
20221205173956_-117.081863_46.781553_bgr.png
20221103163702_-117.081888_46.781508_bgr.png
20221102203326_-117.081888_46.781508_bg

In [5]:
feats_raw = da.vstack(feats_raw).reshape((-1,n_feat)).astype(np.float32)
labels3 = da.vstack(labels_raw).reshape((-1,1)).astype(np.int32).ravel()

In [6]:
feats_raw

dask.array<astype, shape=(61440000, 108), dtype=float32, chunksize=(155344, 108), chunktype=numpy.ndarray>

In [7]:
labels3 = labels3.reshape((-1,1))

In [10]:
train_feats, test_feats, train_labels, test_labels = train_test_split(feats_raw, labels3, test_size=0.2, random_state=42)

In [11]:
train_feats

dask.array<concatenate, shape=(49151900, 108), dtype=float32, chunksize=(124275, 108), chunktype=numpy.ndarray>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#Pipeline
#initial scaling
scaler = StandardScaler()
#Best parameter (CV score=0.892):
#{'clf__activation': 'logistic', 'clf__hidden_layer_sizes': (432, 117, 32), 'pca__n_components': 0.999}

In [ ]:
train_feats = scaler.fit_transform(train_feats)

In [20]:
np.array(train_feats)




/project/nsaru/nsaru-cv/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 49.44 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2023-08-06 15:58:16,073 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/project/nsaru/nsaru-cv/lib/python3.11/site-packages/distributed/protocol/core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/project/nsaru/nsaru-cv/lib/python3.11/site-packages/msgpack/__init__.py", line 38, in packb
    return Packer(**kwargs).pack(o)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cm

type: ('concatenate-50fb3c3a5e8699ec2bf1390a16e639d4', 372, 0)

2023-08-06 15:58:49,954 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
if version=='V1':
    #pca
    pca = PCA(svd_solver='full',n_components=0.999)
    clf = MLPClassifier(max_iter=1000,hidden_layer_sizes=[432,117,32],activation='logistic')
    pipeline = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("clf", clf)])
    pipeline.fit(train_feats,train_labels)
    filename = os.path.join(p3,'model_pipeline_'+version+'_'+model+'_final.pk.sav')
    with open(filename, 'wb') as f:  # Python 3: open(..., 'wb'
        pickle.dump(pipeline.best_estimator_, f)

    pred = pipeline.predict(test_feats)

elif version=='V2':
    #pca
    pca = PCA(svd_solver='full')
    clf = MLPClassifier(max_iter=1000)
    if model == 'mlp':
        #mlp
        clf = MLPClassifier(max_iter=1000)
        #tune hyperparameters
        layers = []

        for layer1 in [2,4,8,16]:
            for layer2 in [2,4]:
                layer = (layer1*n_feat//2, int(np.sqrt(n_feat//2*n_components*layer1*layer2)), layer2*n_components)
                layers.append(layer)

        parameters = {'pca__n_components':(0.99,0.999),'clf__activation':('relu','logistic'),'clf__hidden_layer_sizes':layers}

    elif model == 'hgbc':
        clf = HistGradientBoostingClassifier(max_iter=1000)

        parameters = {'pca__n_components':(0.99,0.999),'clf__min_samples_leaf':(20,200,2000,20000),'clf__max_bins':(31,63,127,255)}

    pipeline = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("clf", clf)])

    search = HalvingGridSearchCV(pipeline, parameters, test_size=.2)#,n_jobs=-1,cv=5)

    search.fit(train_feats, train_labels)
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    filename = os.path.join(p3,'model_pipeline_'+version+'_'+model+'_final.pk.sav')
    with open(filename, 'wb') as f:  # Python 3: open(..., 'wb'
        pickle.dump(search.best_estimator_, f)

    pred = search.predict(test_feats)

In [ ]:
M,f,a = cornfusion(test_labels,pred,n_components)

plt.matshow(M)
plt.ylabel("Predicted")
plt.xlabel("Observed")
plt.title(version+" Confusion Matrix")
plt.savefig(os.path.join(p,'m_'+version+'_'+model+'_final.png'),dpi=300)

print(f,a)

M_df = {}
M_df['sun_soil'] = M[:,0]
M_df['sun_res'] = M[:,1]
M_df['sun_can'] = M[:,2]
M_df['sun_snow'] = M[:,3]
M_df['shade_soil'] = M[:,4]
M_df['shade_res'] = M[:,5]
M_df['shade_can'] = M[:,6]
M_df['shade_snow'] = M[:,7]
M_df = DataFrame(M_df)
M_df.to_csv(os.path.join(p3,'M_'+version+'_'+model+'_final.csv'))

p_df = DataFrame(search.best_params_)
p_df.to_csv(os.path.join(p3,'params_'+version+'_'+model+'_final.csv'))